In [ ]:
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pydicom
from matplotlib.widgets import Slider
from adjustText import adjust_text
from scipy.spatial import Delaunay
from scipy.stats import ttest_rel, wilcoxon, pearsonr
from statsmodels.multivariate.manova import MANOVA
import statsmodels.api as sm
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, classification_report
import napari


## CHECK POINT

In [ ]:

# Load saved files
label_mask = np.load("roi_label_mask.npy")
volume_80kv = np.load("volume_80kv.npy")
volume_150kv = np.load("volume_150kv.npy")
vnca_map = np.load("vnca_map.npy")

# Load HU distribution DataFrame
df_hu = pd.read_csv("hu_distributions_by_roi.csv")

#  load all reference POINTS
point_csvs = glob.glob("points*.csv")
point_data = {fname: pd.read_csv(fname) for fname in point_csvs}


In [ ]:


def load_dicom_volume(folder_path):
    dcm_files = [f for f in os.listdir(folder_path) if f.endswith(".dcm")]
    dcm_files.sort(key=lambda f: int(pydicom.dcmread(os.path.join(folder_path, f)).InstanceNumber))
    slices = [pydicom.dcmread(os.path.join(folder_path, f)).pixel_array.astype(np.int16) for f in dcm_files]
    return np.stack(slices, axis=0)

path_80kv = r"" # Your path to the 80kV DICOM files
path_150kv = r"" # Your path to the Sn150kV DICOM files

volume_80kv = load_dicom_volume(path_80kv)
volume_150kv = load_dicom_volume(path_150kv)

assert volume_80kv.shape == volume_150kv.shape, "Mismatch in shape between 80kV and Sn150kV volumes"

vnca_map = volume_80kv.astype(np.float32) - volume_150kv.astype(np.float32)

slice_idx = volume_80kv.shape[0] // 2
plt.figure(figsize=(18,5))

plt.subplot(1,3,1)
plt.imshow(volume_80kv[slice_idx], cmap='gray')
plt.title("80 kV")
plt.axis("off")

plt.subplot(1,3,2)
plt.imshow(volume_150kv[slice_idx], cmap='gray')
plt.title("Sn150 kV")
plt.axis("off")

plt.subplot(1,3,3)
plt.imshow(vnca_map[slice_idx], cmap='hot')
plt.title("VNCa Map (approx)")
plt.axis("off")

plt.tight_layout()
plt.show()


In [ ]:
%matplotlib widget

vmin = np.min(vnca_map)
vmax = np.max(vnca_map)
absmax = max(abs(vmin), abs(vmax)) 

slice_idx = volume_80kv.shape[0] // 2

fig, axes = plt.subplots(1, 3, figsize=(18, 6))
plt.subplots_adjust(bottom=0.2)

im1 = axes[0].imshow(volume_80kv[slice_idx], cmap='gray')
axes[0].set_title("80 kV")
axes[0].axis("off")

im2 = axes[1].imshow(volume_150kv[slice_idx], cmap='gray')
axes[1].set_title("Sn150 kV")
axes[1].axis("off")

im3 = axes[2].imshow(vnca_map[slice_idx], cmap='bwr', vmin=-absmax, vmax=absmax)
axes[2].set_title("VNCa Map (bwr)")
axes[2].axis("off")

ax_slider = plt.axes([0.25, 0.05, 0.5, 0.03])
slider = Slider(ax_slider, 'Slice', 0, volume_80kv.shape[0] - 1, valinit=slice_idx, valstep=1)

def update(val):
    idx = int(slider.val)
    im1.set_data(volume_80kv[idx])
    im2.set_data(volume_150kv[idx])
    im3.set_data(vnca_map[idx])
    fig.canvas.draw_idle()

slider.on_changed(update)

plt.show()


In [ ]:
viewer = napari.Viewer()

viewer.add_image(vnca_map, name="VNCa", colormap='bwr')

roi_labels = np.zeros_like(vnca_map, dtype=np.uint8)
label_layer = viewer.add_labels(roi_labels, name="ROI Mask")

napari.run()  

## Re-view the 3D Masks/Points

In [ ]:
viewer = napari.Viewer()

vnca_map = np.load("vnca_map.npy")
viewer.add_image(vnca_map, name="VNCa", colormap='bwr')

roi_labels = np.zeros_like(vnca_map, dtype=np.uint8)
viewer.add_labels(roi_labels, name="ROI Mask")

point_files = glob.glob("points*.csv")
for file in point_files:
    points_df = pd.read_csv(file)
    points = points_df[["Z", "Y", "X"]].values  # Napari expects (n_points, 3) array
    viewer.add_points(points, name=file.replace(".csv", ""), size=5, face_color='yellow')

napari.run()

# ROI1: 9
# ROI2: 8
# ROI3: 7
# ROI4: 3
# ROI5: 5
# ROI6: 2
# ROI7: 1
# ROI8: 4


In [ ]:
points1 = viewer.layers['points1'].data 


In [ ]:

Z, Y, X = vnca_map.shape
zz, yy, xx = np.meshgrid(np.arange(Z), np.arange(Y), np.arange(X), indexing='ij')
all_voxel_coords = np.column_stack([zz.ravel(), yy.ravel(), xx.ravel()])

label_mask = np.zeros((Z, Y, X), dtype=np.uint8)


try:
    viewer = napari.current_viewer()
    if viewer is None:
        raise ValueError
except:
    viewer = napari.Viewer()

point_layers = [layer for layer in viewer.layers if layer.name.startswith('points')]
point_layers.sort(key=lambda l: l.name)

for i, layer in enumerate(point_layers, 1):  
    pts = layer.data
    if pts.shape[0] != 8:
        print(f"Skipping {layer.name}: expected 8 points, found {pts.shape[0]}")
        continue

    try:
        hull = Delaunay(pts)
        inside = hull.find_simplex(all_voxel_coords) >= 0
        mask = inside.reshape((Z, Y, X))
        label_mask[mask] = i  
        print(f"Added ROI from {layer.name} as label {i}")
    except Exception as e:
        print(f"Failed to create mask from {layer.name}: {e}")
        
viewer.add_labels(label_mask, name="3D ROI Labels")
napari.run()

In [ ]:
if 'ROI Mask' in viewer.layers:
    viewer.layers['ROI Mask'].data = label_mask
else:
    viewer.add_labels(label_mask, name='ROI Mask')


In [ ]:
for i in range(1, len(point_layers)+1):
    values = vnca_map[label_mask == i]
    print(f"ROI {i}: mean HU = {values.mean():.2f}, std = {values.std():.2f}, n = {len(values)}")


## Distribution 

In [ ]:
roi_labels = np.unique(label_mask)
roi_labels = roi_labels[roi_labels > 0]

plt.figure(figsize=(6, 12))  # taller figure for vertical layout

for i, label in enumerate(roi_labels, 1):
    vals_80 = volume_80kv[label_mask == label]
    vals_150 = volume_150kv[label_mask == label]
    vals_vnc = vnca_map[label_mask == label]

    plt.subplot(3, 1, 1)
    plt.hist(vals_80, bins=100, alpha=0.4, label=f"ROI {label}")
    plt.title("80 kV HU Distribution")
    plt.xlabel("HU")
    plt.ylabel("Count")

    plt.subplot(3, 1, 2)
    plt.hist(vals_150, bins=100, alpha=0.4, label=f"ROI {label}")
    plt.title("Sn150 kV HU Distribution")
    plt.xlabel("HU")
    plt.ylabel("Count")

    plt.subplot(3, 1, 3)
    plt.hist(vals_vnc, bins=100, alpha=0.4, label=f"ROI {label}")
    plt.title("VNCa Approx HU Distribution")
    plt.xlabel("HU")
    plt.ylabel("Count")

# Add legends
for j in range(1, 4):
    plt.subplot(3, 1, j)
    plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
np.save("roi_label_mask.npy", label_mask)
np.save("volume_80kv.npy", volume_80kv)
np.save("volume_150kv.npy", volume_150kv)
np.save("vnca_map.npy", vnca_map)
for layer in viewer.layers:
    if layer.name.startswith("points"):
        np.savetxt(f"{layer.name}.csv", layer.data, delimiter=",", header="Z,Y,X", comments="")

data = []
roi_labels = np.unique(label_mask)
roi_labels = roi_labels[roi_labels > 0]  

for label in roi_labels:
    mask = label_mask == label
    for source_name, volume in zip(["80kv", "150kv", "vnca"], [volume_80kv, volume_150kv, vnca_map]):
        values = volume[mask]
        for v in values:
            data.append({"ROI": int(label), "Source": source_name, "HU": float(v)})
import pandas as pd
df = pd.DataFrame(data)
df.to_csv("hu_distributions_by_roi.csv", index=False)



In [ ]:
roi_labels = np.unique(label_mask)
roi_labels = roi_labels[roi_labels > 0]

plt.figure(figsize=(8, 6))

for label in roi_labels:
    x_vals = volume_80kv[label_mask == label].flatten()
    y_vals = volume_150kv[label_mask == label].flatten()
    
    plt.scatter(x_vals, y_vals, s=5, alpha=0.3, label=f"ROI {label}")

plt.title("")
plt.xlabel("80kV HU")
plt.ylabel("Sn150kV HU")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


## Add Metadata (Water/Glycerol/Barium contents)

In [ ]:
volume_80kv_hu = volume_80kv - 1024
volume_150kv_hu = volume_150kv - 1024

roi_metadata = {
    1: {"water": 95.95, "glycerol": 0, "barium": 1.0},
    2: {"water": 96.45, "glycerol": 0, "barium": 0.5},
    3: {"water": 96.85, "glycerol": 0, "barium": 0.1},
    4: {"water": 77.45, "glycerol": 15, "barium": 0},
    5: {"water": 62.45, "glycerol": 30, "barium": 0},
    6: {"water": 82.45, "glycerol": 10, "barium": 0},
    7: {"water": 87.45, "glycerol": 5,  "barium": 0},
    8: {"water": 72.45, "glycerol": 20, "barium": 0},
}


hu_data = []

for label in roi_labels:
    x_vals = volume_80kv_hu[label_mask == label].flatten()
    y_vals = volume_150kv_hu[label_mask == label].flatten()
    water = roi_metadata[label]["water"]
    glycerol = roi_metadata[label]["glycerol"]
    barium = roi_metadata[label]["barium"]

    for x, y in zip(x_vals, y_vals):
        hu_data.append({
            "ROI": label,
            "HU_80kV": x,
            "HU_150kV": y,
            "Water_Content": water,
            "Glycerol_Content": glycerol,
            "Barium_Content": barium
        })

hu_df = pd.DataFrame(hu_data)




In [ ]:

plt.figure(figsize=(8, 6))
sc = plt.scatter(hu_df["HU_80kV"], hu_df["HU_150kV"],
                 c=hu_df["Water_Content"], cmap="coolwarm", s=3, alpha=0.3)
plt.colorbar(sc, label="Water %")
plt.xlabel("80kV HU")
plt.ylabel("Sn150kV HU")
plt.title("HU Distribution Colored by Water Content")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))
sc = plt.scatter(hu_df["HU_80kV"], hu_df["HU_150kV"],
                 c=hu_df["Glycerol_Content"], cmap="coolwarm", s=3, alpha=0.3)
plt.colorbar(sc, label="Glycerol %")
plt.xlabel("80kV HU")
plt.ylabel("Sn150kV HU")
plt.title("HU Distribution Colored by Glycerol Content")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:
plt.figure(figsize=(8, 6))

sc = plt.scatter(
    hu_df["HU_80kV"], hu_df["HU_150kV"],
    c=hu_df["Barium_Content"],
    cmap="coolwarm",
    s=3,
    alpha=0.3
)
plt.colorbar(sc, label="Barium Sulfate %")
plt.xlabel("80kV HU")
plt.ylabel("Sn150kV HU")
plt.title("HU Distribution Colored by Barium Sulfate Content")
plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:

plt.figure(figsize=(8, 6))

sc = plt.scatter(
    hu_df["HU_80kV"], hu_df["HU_150kV"],
    c=hu_df["Water_Content"],
    cmap="coolwarm",
    s=3,
    alpha=0.3
)
plt.colorbar(sc, label="Water %")
plt.xlabel("80kV HU")
plt.ylabel("Sn150kV HU")
plt.title("HU Distribution Colored by Water Content")

texts = []
for roi in roi_labels:
    sub = hu_df[hu_df["ROI"] == roi]
    x_median = sub["HU_80kV"].median()
    y_median = sub["HU_150kV"].median()
    water = sub["Water_Content"].iloc[0]
    glycerol = sub["Glycerol_Content"].iloc[0]
    barium = sub["Barium_Content"].iloc[0]
    label = f"{water:.1f}% H₂O\n{glycerol:.1f}% Gly\n{barium:.1f}% BaSO₄"
    texts.append(
        plt.text(
            x_median, y_median, label,
            fontsize=8,
            ha='center',
            va='center',
            bbox=dict(facecolor='white', alpha=0.6, edgecolor='none', boxstyle='round,pad=0.3')
        )
    )

adjust_text(texts, arrowprops=dict(arrowstyle='->', color='gray', lw=0.5))

plt.grid(True)
plt.tight_layout()
plt.show()


In [ ]:


vals_80kv_real = volume_80kv - 1024
vals_150kv_real = volume_150kv - 1024

roi_labels = np.unique(label_mask)
roi_labels = roi_labels[roi_labels > 0]

roi_metadata = {
    1: {"water": 95.95, "glycerol": 0, "barium": 1.0},
    2: {"water": 96.45, "glycerol": 0, "barium": 0.5},
    3: {"water": 96.85, "glycerol": 0, "barium": 0.1},
    4: {"water": 77.45, "glycerol": 15, "barium": 0},
    5: {"water": 62.45, "glycerol": 30, "barium": 0},
    6: {"water": 82.45, "glycerol": 10, "barium": 0},
    7: {"water": 87.45, "glycerol": 5,  "barium": 0},
    8: {"water": 72.45, "glycerol": 20, "barium": 0},
}

plt.figure(figsize=(6, 12))  

for i, label in enumerate(roi_labels, 1):
    mask = label_mask == label
    vals_80 = vals_80kv_real[mask]
    vals_150 = vals_150kv_real[mask]
    
    water = roi_metadata[label]["water"]
    glycerol = roi_metadata[label]["glycerol"]
    barium = roi_metadata[label]["barium"]
    annot = f"ROI {label}\n{water:.1f}% H₂O\n{glycerol:.1f}% Gly\n{barium:.1f}% BaSO₄"

    plt.subplot(3, 1, 1)
    plt.hist(vals_80, bins=100, alpha=0.4, label=annot)

    plt.subplot(3, 1, 2)
    plt.hist(vals_150, bins=100, alpha=0.4, label=annot)

# Titles and axis labels
plt.subplot(3, 1, 1)
plt.title("80 kV HU Distribution")
plt.xlabel("HU")
plt.ylabel("Count")

plt.subplot(3, 1, 2)
plt.title("Sn150 kV HU Distribution")
plt.xlabel("HU")
plt.ylabel("Count")


for j in range(1, 4):
    plt.subplot(3, 1, j)
    plt.legend(
        fontsize=7,
        ncol=4,                    
        loc="upper center",           
        bbox_to_anchor=(0.5, -0.25),   
        frameon=False,
        handlelength=1.5
    )

plt.tight_layout()
plt.show()


## Statistical Test to show separability


In [ ]:

subset = hu_df.sample(n=10000, random_state=42)  
X_80 = subset[["HU_80kV"]]
X_150 = subset[["HU_150kV"]]
X_both = subset[["HU_80kV", "HU_150kV"]]
y = subset["ROI"]


In [ ]:
clf = make_pipeline(StandardScaler(), LogisticRegression(multi_class='multinomial', max_iter=1000))

print("80kV only:")
scores_80 = cross_val_score(clf, X_80, y, cv=5)
print(scores_80.mean())

print("150kV only:")
scores_150 = cross_val_score(clf, X_150, y, cv=5)
print(scores_150.mean())

print("Both combined:")
scores_both = cross_val_score(clf, X_both, y, cv=5)
print(scores_both.mean())



In [ ]:

clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, random_state=42))
print("80kV only:")
scores_80 = cross_val_score(clf, X_80, y, cv=5)
print(scores_80.mean())

print("150kV only:")
scores_150 = cross_val_score(clf, X_150, y, cv=5)
print(scores_150.mean())

print("Both combined:")
scores_both = cross_val_score(clf, X_both, y, cv=5)
print(scores_both.mean())



In [ ]:


clf = make_pipeline(StandardScaler(), RandomForestClassifier(n_estimators=100, random_state=42))

scores_dict = {
    '80kV': cross_val_score(clf, X_80, y, cv=10),
    '150kV': cross_val_score(clf, X_150, y, cv=10),
    'Both': cross_val_score(clf, X_both, y, cv=10)
}

df_scores = pd.DataFrame(scores_dict).melt(var_name="Model", value_name="Accuracy")

plt.figure(figsize=(7, 5))
sns.boxplot(x="Model", y="Accuracy", data=df_scores, palette="Set2")
sns.swarmplot(x="Model", y="Accuracy", data=df_scores, color=".25", size=3)
plt.title("Cross-Validation Accuracy Comparison")
plt.ylabel("Accuracy")
plt.tight_layout()
plt.show()


In [ ]:

print("Wilcoxon (80kV vs Both):", wilcoxon(scores_dict["80kV"], scores_dict["Both"]))
print("Wilcoxon (150kV vs Both):", wilcoxon(scores_dict["150kV"], scores_dict["Both"]))
print("Wilcoxon (80kV vs 150kV):", wilcoxon(scores_dict["80kV"], scores_dict["150kV"]))


In [ ]:
lda = LinearDiscriminantAnalysis(n_components=2)
X_lda = lda.fit_transform(X_both, y)

plt.figure(figsize=(8, 6))
sns.scatterplot(x=X_lda[:, 0], y=X_lda[:, 1], hue=y, palette='tab10', s=10, alpha=0.5)
plt.title("LDA Projection of ROIs (80kV + 150kV)")
plt.xlabel("LD1")
plt.ylabel("LD2")
plt.legend(title="ROI", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.tight_layout()
plt.show()


In [ ]:


manova_df = hu_df[["HU_80kV", "HU_150kV", "ROI"]]
manova_df["ROI"] = manova_df["ROI"].astype(str)

maov = MANOVA.from_formula('HU_80kV + HU_150kV ~ ROI', data=manova_df)
print(maov.mv_test())


In [ ]:


# Group HU by mean ROI values
summary_df = hu_df.groupby("ROI")[["HU_80kV", "HU_150kV", "Water_Content", "Glycerol_Content", "Barium_Content"]].mean()

# Correlation with water
print("80kV vs Water:", pearsonr(summary_df["HU_80kV"], summary_df["Water_Content"]))
print("150kV vs Water:", pearsonr(summary_df["HU_150kV"], summary_df["Water_Content"]))

# Correlation with glycerol
print("80kV vs Glycerol:", pearsonr(summary_df["HU_80kV"], summary_df["Glycerol_Content"]))
print("150kV vs Glycerol:", pearsonr(summary_df["HU_150kV"], summary_df["Glycerol_Content"]))

# Correlation with barium
print("80kV vs Barium:", pearsonr(summary_df["HU_80kV"], summary_df["Barium_Content"]))
print("150kV vs Barium:", pearsonr(summary_df["HU_150kV"], summary_df["Barium_Content"]))


In [ ]:
for component in ["Water_Content", "Glycerol_Content", "Barium_Content"]:
    X = sm.add_constant(summary_df[[component]])
    model = sm.OLS(summary_df["HU_80kV"], X).fit()
    print(f"80kV vs {component}:\n", model.summary())



In [ ]:
for component in ["Water_Content", "Glycerol_Content", "Barium_Content"]:
   
    X = sm.add_constant(summary_df[[component]])
    model = sm.OLS(summary_df["HU_150kV"], X).fit()
    print(f"150kV vs {component}:\n", model.summary())

In [ ]:


# Mean HU per ROI
summary_df = hu_df.groupby("ROI")[["HU_80kV", "HU_150kV", "Water_Content", "Glycerol_Content", "Barium_Content"]].mean()

X = summary_df[["Water_Content", "Glycerol_Content", "Barium_Content"]]
X_const = sm.add_constant(X)

model_80 = sm.OLS(summary_df["HU_80kV"], X_const).fit()
model_150 = sm.OLS(summary_df["HU_150kV"], X_const).fit()

coef_df = pd.DataFrame({
    "80kV": model_80.params.drop("const"),
    "Sn150kV": model_150.params.drop("const")
})


In [ ]:


plt.figure(figsize=(5, 3))
sns.heatmap(coef_df.T, annot=True, cmap="coolwarm", center=0, cbar_kws={"label": "Regression Coefficient"})
plt.title("HU Sensitivity to Biochemical Composition")
plt.xlabel("Biochemical Component")
plt.ylabel("Energy Level")
plt.tight_layout()
plt.show()


In [ ]:
coef_df_plot = coef_df.reset_index().melt(id_vars="index", var_name="Energy", value_name="Coefficient")
coef_df_plot.rename(columns={"index": "Component"}, inplace=True)

plt.figure(figsize=(6, 4))
sns.barplot(data=coef_df_plot, x="Component", y="Coefficient", hue="Energy")
plt.axhline(0, color='black', lw=1)
plt.title("HU Sensitivity by Energy Level")
plt.tight_layout()
plt.show()
